Tester et entraîner un modèle de reconnaissance d'écriture
===
<br>

Les dossiers qui constituent l'archive de la correspondance de Constance de Salm réunissent des documents écrits par plusieurs scribes. La différence des écritures représente une difficulté certaine pour la reconnaissance automatique. Ce *notebook* explique comment procéder à la reconnaissance automatique de l'écriture dans ce contexte particulier.

# Classer les images par mains

Inventorier toutes les mains attestées dans une source n'est pas toujours aisé. L'objectif reste avant tout de **repérer les mains principales** : celles attestées sur le plus grand nombre de pages. Une main qui ne serait attestée que sur une dizaine de pages ne mériterait pas d'être classée, car entraîner un modèle de reconnaissance d'écriture pour un petit nombre de pages serait une perte de temps.

Voici un modèle d'arborescence pour le rangement des fichiers et le nommage des dossiers. Il convient de copier les fichiers correspondant à chaque main identifiée dans un dossier propre, **sans supprimer du dossier img-complet/** ces fichiers lors de leur copie :

```txt
.
└── entrainements
    ├── img-complet
    │   ├── CdS02_Konv019_0011.jpg
    │   ├── CdS02_Konv019_0012.jpg
    │   ├── CdS02_Konv019_0013.jpg
    │   ├── …
    │   ├── CdS02_Konv019_0033.jpg
    │   ├── CdS02_Konv019_0037.jpg
    │   ├── CdS02_Konv019_0039.jpg
    │   └── …
    │
    ├── mainCdS02_Konv019_01
    │   ├── CdS02_Konv019_0011.jpg
    │   ├── CdS02_Konv019_0012.jpg
    │   ├── CdS02_Konv019_0013.jpg
    │   └── …
    │
    └── mainCdS02_Konv019_02
        ├── CdS02_Konv019_0033.jpg
        ├── CdS02_Konv019_0037.jpg
        ├── CdS02_Konv019_0039.jpg
        └── …
```

Il est important qu'aucune image ne soit placée dans deux dossiers de main différents, même si les deux mains en questions sont toutes deux attestées dans l'image. Le script suivant permet de retourner la **liste des doublons éventuels** :

In [ ]:
import os

# On initie un dictionnaire pour la récupération des noms de fichiers et de leur racine
dictFichiers = {}

# On analyse l'arborescence courante
for racine, dossiers, fichiers in os.walk("./entrainements/"):
    # On boucle sur les fichiers
    for fichier in fichiers:
        # On ne sélectionne que les fichiers .jpg et les fichiers rangés dans des dossiers de main
        if fichier[-3:] == "jpg" and racine[:20] == "./entrainements/main":
            # Si le fichier est absent du dictionnaire, on l'ajoute
            if not dictFichiers.get(fichier):
                dictFichiers[fichier] = [racine]
            # Si le fichier est présent dans le dictionnaire, on ajoute sa racine à la liste des valeurs
            else:
                dictFichiers[fichier].append(racine)
    
# On initie un booléen pour confirmer l'absence de doulon
doublons = False

# Pour rechercher les doublons, on boucle sur les entrées du dictionnaire
for fichier in dictFichiers:
    # Si le nom du fichier est associé à une liste de plus de 1 racine
    if len(dictFichiers[fichier]) > 1:
        doublons = True
        # On retourne alors un message d'alerte
        print(f"Le fichier {fichier} est doublonné :")
        for racine in dictFichiers[fichier]:
            print(f"{racine}/{fichier}")

# On retourne un message si aucun doublon n'a été trouvé
if not doublons:
    print("Bravo ! Aucun doublon n'a été trouvé.")

S'il existe des doublons, il est important de **les supprimer**. On peut le faire automatiquement grâce à la commande suivante, qui va supprimer arbitrairement la première occurrence :

In [ ]:
# On boucle sur les entrées du dictionnaire
for fichier in dictFichiers:
    # Si le nom du fichier est associé à une liste de plus de 1 racine
    if len(dictFichiers[fichier]) > 1:
        # On supprime le premier de la liste
        !rm {dictFichiers[fichier][0]}/{fichier}

# Créer un échantillon-test de chaque écriture

Afin de pouvoir tester un ou plusieurs modèles, il est nécessaire de constituer une vérité de terrain de 2-3 doubles pages (selon la densité d'écriture qu'elles contiennent).

## Critères de sélection

On crée dans le dossier de chaque main un dossier **test/** contenant des spécimens d'écriture selon les critères suivants :
- Reproductions de bonne qualité (sans problème de transparence)
- Pages choisies de manière discontinue (l'écriture d'une même main peut en effet varier et il est utile de prendre en compte cette variété pour le test)

Si certaines mains ne sont attestées qu'en compagnie d'autres écritures, on veillera à limiter le test de reconnaissance d'écriture aux seuls lignes de la main à tester (voir *infra*).

## Organisation des fichiers

Voici un modèle d'arborescence pour le rangement et le nommage des fichiers et dossiers. Il est important de **déplacer les fichiers-images** choisis vers le dossier test/ et non **pas les copier** :
```txt
.
└── entrainements
    ├── img-complet
    │   ├── CdS02_Konv019_0011.jpg
    │   ├── CdS02_Konv019_0012.jpg
    │   ├── CdS02_Konv019_0013.jpg
    │   ├── …
    │   ├── CdS02_Konv019_0033.jpg
    │   ├── CdS02_Konv019_0037.jpg
    │   ├── CdS02_Konv019_0039.jpg
    │   └── …
    │
    ├── mainCdS02_Konv019_01
    │   ├── CdS02_Konv019_0013.jpg
    │   ├── …
    │   ├── test
    │   │   ├── CdS02_Konv019_0011.jpg
    │   │   └── CdS02_Konv019_0012.jpg
    │   └── train
    │
    └── mainCdS02_Konv019_02
        ├── CdS02_Konv019_0039.jpg
        ├── …
        ├── test
        │   ├── CdS02_Konv019_0033.jpg
        │   └── CdS02_Konv019_0037.jpg
        └── train
```

## Segmenter et annoter les pages

On se reporte pour cela au [notebook dédié](./Segmenter_et_annoter_une_page.ipynb).

## Transcrire

Constituer un spécimen de test ou une collection d'entraînement suppose de ne travailler que **sur une seule main**, en choisissant soigneusement des pages où une seule main est attestée. Mais si l'on a été obligé de prendre une page où plusieurs mains sont attestées, il faut **veiller à ne pas transcrire les parties d'une page dont l'écriture ne serait pas l'objet du test**.

### Suivre les règles de transcription

On suit de manière rigoureuse les préconisations pour la transcription énoncées dans la [documentation analytique](https://github.com/sbiay/CdS-edition/blob/main/documentation/documentation.pdf) (cf. annexe Normes de transcription).

### Exporter au format XML-Alto

Si l'on transcrit les pages sous e-Scriptorium, on peut se référer au tutoriel Lectaurep pour [exporter les fichiers](https://lectaurep.hypotheses.org/documentation/prendre-en-main-escriptorium#export) une fois le travail fini.

### Ranger les fichiers Alto

On place les transcriptions au format Alto avec les images classées dans l'arborescence :

```txt
.
└── entrainements
    ├── img-complet
    │   └── …
    │
    ├── mainCdS02_Konv019_01
    │   ├── CdS02_Konv019_0013.jpg
    │   ├── …
    │   ├── test
    │   │   ├── CdS02_Konv019_0011.jpg
    │   │   ├── CdS02_Konv019_0011.xml <=
    │   │   ├── CdS02_Konv019_0011.jpg
    │   │   └── CdS02_Konv019_0012.xml <=
    │   └── train
    │
    └── mainCdS02_Konv019_02
        ├── CdS02_Konv019_0039.jpg
        ├── …
        ├── test
        │   ├── CdS02_Konv019_0033.jpg
        │   ├── CdS02_Konv019_0033.xml <=
        │   ├── CdS02_Konv019_0037.jpg
        │   └── CdS02_Konv019_0037.xml <=
        └── train
```

## Éliminer les lignes parasites

Si une page comporte des lignes vides (c'est le cas si plusieurs écritures sont attestées sur la même page, on ne transcrit que l'écriture à tester), le script [supprLignesVides.py](https://github.com/sbiay/CdS-edition/blob/main/htr/py/supprLignesVides.py) permet de les éliminer :

In [ ]:
!python3 py/supprLignesVides.py CHEMIN-DE-FICHIER

# Tester des modèles HTR

## Installer l'application Kraken

On recommande pour tester un modèle d'utiliser l'application Kraken en ligne de commande, disponible pour Linux et Mac OSX (non pour Windows). Les instructions sont consultables [ici](https://github.com/mittagessen/kraken#installation).

## Importer un modèle

Les modèles extérieurs au projet que l'on a utilisés sont téléchargeables sur le [Gitlab du laboratoire Inria](https://gitlab.inria.fr/dh-projects/kraken-models/-/tree/master/transcription%20models) :
- **generic_lectaurep_26.mlmodel** : modèle mixte, entraîné *from scratch* à partir d'une variété d'écritures administratives du XIXe siècle (LECTAUREP)
- **cm_ft_mrs15_11.mlmodel** : entraîné en affinant (44 pages) sur les contrats de mariage le modèle **mixte_mrs_15**, lui -même entraîné *from scratch* à partir d'une varété d'écritures administratives du XIXe siècle (LECTAUREP)

Les modèles propres au projet sont :
- **souvay.mlmodel** : entraîné *from scratch* à partir d'un petit nombre de vérités de terrain [source](https://github.com/dhi-digital-humanities/constance-de-salm/blob/main/HTR/Training%20Models/CdS02_Konv002-02-03/Models/model_best.mlmodel)
- **cds_lectcm_04_mains_01.mlmodel** : entraîné en affinant (40ne de pages) à partir du modèle **cm_ft_mrs15_11.mlmodel**

Lors du traitement d'une nouvelle source, il est recommandé de partir du modèle cds_lectcm_*.mlmodel entraîné sur le nombre de "mains" le plus élevé.


## Initier un journal de tests

Le script [journalReconn.py](./py/journalReconn.py) permet de pré-remplir un journal pour l'enregistrement des résultats des tests effectués sur les modèles. 

On lui donne comme argument un nom de modèle et il écrit dans le fichier Json [journal-rec.json](./entrainements/journal-rec.json) :
- A la date et à l'heure courante…
- Pour chaque main possédant un dossier dans l'arborescence de fichiers (./entrainements/main*/)…
- Les noms de ces mains…
- Et prépare le renseignement des valeurs de test.

Conseils d'utilisation :
- Paramètre MODELE : obligatoire ; renseigner un **nom de modèle** plutôt que son chemin relatif ou absolu
- Option -n, --no-ground-truth : le test initial se fait avec cette option, sans avoir entraîné le modèle sur les vérités de terrain
- Option -i, --ignore : pour ignorer une main (prend comme argument le nom de son dossier)

In [ ]:
# python3 py/journalReconn.py MODELE -n (pour un test initial sans entraînement)
!python3 py/journalReconn.py cds_lectcm_04_mains_01.mlmodel -n

Le contenu écrit dans le fichier journal donne comme **accuracy** pour chaque main une valeur de 0. Cette valeur doit être saisie manuellement dans le fichier une fois effectué le test comme suit.


## Effectuer un test

Avec la commande suivante on effectue un test d'acuité pour un modèle sur une main particulière (on doit relever l'*Average accuracy*) :

In [ ]:
# ketos test -m ./entrainements/modeles-rec/NOM-MODEL.mlmodel ./entrainements/NOM-MAIN/test/*xml -f alto
!ketos test -m ./entrainements/modeles-rec/cm_ft_mrs15_11.mlmodel ./entrainements/mainCdS02_Konv019_02/test/*xml -f alto

Il convient de choisir le modèle présentant **les meilleurs résultats pour l'ensemble des écritures testées**. Si le meilleur modèle n'atteint pas 90% pour tout ou partie des mains, il convient de l'entraîner par l'apport de vérités de terrain pour chaque main n'atteignant pas ce score.

# Entraîner un modèle

## Principe général

L'entraînement d'un modèle multi-main consiste à constituer une vérité de terrain pour chaque main concernée et à **entraîner un modèle sur l'ensemble** de ces vérités de terrain réunies.

## Constituer des collections d'entraînement

On procède à la constitution d'une vérité de terrain pour chaque spécimen d'écriture. Il s'agit de transcrire l'**équivalent d'une dizaine de pages simples** : si les pages présentent des blancs importants ou d'autres écritures étrangères à l'entraînement (elles ne doivent pas être transcrites) on augmentera le nombre de pages pour parvenir grosso modo à ce volume de 10 pages simples.

La transcription de ces pages se fait à la main, sans l'aide d'une première reconnaissance de l'écriture, car il est plus compliqué de corriger une mauvaise prédiction que de transcrire manuellement. On reprend pour cela la [démarche](http://localhost:8888/notebooks/Tester_et_entrainer_un_modele_HTR_avec_Kraken.ipynb#Cr%C3%A9er-un-%C3%A9chantillon-test-de-chaque-%C3%A9criture) énoncée plus haut pour la création d'un échantillon test.

## Lancer l'entraînement

Une fois les collections d'entraînement de chaque main, on écrit dans un fichier dédié les chemins de fichiers des vérités de terrain sur lesquelles l'entraînement doit s'appuyer (i.e. tous les fichiers XML contenus dans les dossiers **train/**) :

In [ ]:
# On écrit la liste des chemins de fichiers
!find -wholename */train/*xml > train.txt

# On crée un dossier de sortie pour le modèle entraîné
!mkdir --parents ./entrainements/modeles-rec/out/

# La commande suivante entraîne le modèle cm_ft_mrs15_11.mlmodel
!ketos train -r 0.0001 --lag 20  -s '[1,120,0,1 Cr3,13,32 Do0.1,2 Mp2,2 Cr3,13,32 Do0.1,2 Mp2,2 Cr3,9,64 Do0.1,2 Mp2,2 Cr3,9,64 Do0.1,2 S1(1x0)1,3 Lbx200 Do0.1,2 Lbx200 Do.1,2 Lbx200 Do]' --augment --device cpu --resize add -i ./entrainements/modeles-rec/cm_ft_mrs15_11.mlmodel -t ./entrainements/train.txt -f alto --output --output ./entrainements/modeles-rec/out/

## Tester le nouveau modèle

Une fois le modèle entraîné, on renouvelle la procédure de test expliquée ci-dessus.

En résumé on initie les résultats dans le journal d'entraînement :

In [ ]:
!python3 py/journalReconn.py MODELE

On teste le modèle sur les différentes mains :

In [ ]:
# ketos test -m ./entrainements/modeles-rec/NOM-MODEL.mlmodel ./entrainements/NOM-MAIN/test/*xml -f alto
!ketos test -m ./entrainements/modeles-rec/cm_ft_mrs15_11.mlmodel ./entrainements/mainCdS02_Konv019_02/test/*xml -f alto

On complète manuellement le journal d'entraînement avec les valeurs d'*accuracy*.

### En cas de résultat insatisfaisant

**Si certaines mains ont toujours un score inférieur à 90%**, on réitère la procédure d'entraînement en apportant 5 nouvelles pages pour chaque main n'atteignant pas le score souhaité.

On repart du **modèle de départ** du précédent entraînement et non du modèle de sortie : en somme, l'entraînement doit repartir de 0.

### En cas de résultat satisfaisant

On copie l'ensemble des fichiers d'entraînement dans un dossier dédié aux vérités de terrain du projet et les fichiers de test dans un dossier distinct :

In [ ]:
!mkdir --parents ./verite-terrain/
!find -wholename "*/train/*" | xargs -I '{}' cp '{}' ./verite-terrain/
!mkdir --parents ./test-rec/
!find -wholename "*/test/*" | xargs -I '{}' cp '{}' ./test-rec/

On rassemble les informations relatives aux mains attestées par ces vérités de terrain et ces tests au moyen du script suivant :

In [ ]:
import json
from py.constantes import TRAITNTENCOURS

# On ouvre le fichier mains.json du dossier en cours de traitement
with open(f"{TRAITNTENCOURS}mains.json") as jsonf:
    traitnt = json.load(jsonf)

# On récupère le contenu du fichier de synthèse des mains s'il existe
with open("./mains/mains.json") as jsonf:
    synthese = json.load(jsonf)

# On trie les mains par ordre alpha-numérique
labelsMains = []
for main in traitnt:
    labelsMains.append(main)
tri = sorted(labelsMains)

# On boucle sur chaque main du fichier du traitement courant
for main in tri:
    # Si la main n'existe pas encore dans la synthèse
    if not synthese.get(main):
        # On l'ajoute intégralement
        synthese[main] = traitnt[main]
    # Si la main existe dans la synthèse
    else:
        # On boucle sur chaque type de fichier
        for type in traitnt[main]:
            # On boucle sur chaque fichier du traitement en cours
            for fichier in traitnt[main][type]:
                # Si le fichier du traitement en cours est présent dans la synthèse
                if fichier not in synthese[main][type]:
                    # on l'ajoute à la liste des fichiers de la synthèse
                    synthese[main][type].append(fichier)

# On écrit une nouvelle version du fichier de synthèse
with open("./mains/mains.json", mode="w") as jsonf:
    json.dump(synthese, jsonf, indent=3)
    print(f"Le fichier ./mains/mains.json a été correctement enrichi.")

On renomme le **meilleur modèle** à partir de la suggestion de nommage (`label_output`) indiquée dans le fichier **journal-rec.json** du traitement en cours et on le place dans le dossier **./modeles-rec**

# Effectuer la prédiction

On [importe le nouveau modèle dans e-Scriptorium](https://lectaurep.hypotheses.org/documentation/prendre-en-main-escriptorium#import_model), puis on lance la prédiction à l'aide de celui-ci.

Une fois la prédiction terminée, on sélectionne l'ensemble du dossier et on l'exporte au format Alto, sans inclure les images, en sélectionnant bien le contenu textuel produit par le modèle de reconnaissance utilisé.

On place les prédictions dans le dossier dédié, que la commande suivante permet de créer s'il n'existe pas déjà :

In [ ]:
from py.constantes import XMLaCORRIGER
# Si le dossier n'a pas encore été créé, on peut lancer la commande suivante pour le faire
!mkdir --parents {XMLaCORRIGER}

print(f"Le dossier {XMLaCORRIGER} est prêt à recevoir les prédictions.")

Le script [injectTranscript.py](https://github.com/sbiay/CdS-edition/blob/main/htr/py/injectTranscript.py) permet de récupérer les transcriptions placées dans les dossiers **./test-rec/** et **./verite-terrain/** et de réinjecter leur contenu dans les prédictions que l'on vient d'exporter. On le lance par la commande suivante :

In [ ]:
!python3 py/injectTranscript.py

Une fois cette opération effectuée, on peut passer à la correction automatisée des prédictions, documentée dans le *notebook* [Corriger une prédiction HTR](./Corriger_une_prediction.ipynb)